In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("ps4_data.csv")

ID3

In [5]:
def entropy(D):
    class_counts = D.iloc[:, -1].value_counts()
    probs = class_counts / len(D)
    return -np.sum(probs * np.log2(probs))

def info_gain(D, attribute):
    total_entropy = entropy(D)
    values = D[attribute].unique()
    weighted_entropy_sum = 0
    for value in values:
        subset = D[D[attribute] == value]
        weighted_entropy_sum += (len(subset) / len(D)) * entropy(subset)
    return total_entropy - weighted_entropy_sum

In [6]:
# ID3 - Decision tree using information gain
def id3(D, attributes):
    if len(D.iloc[:, -1].value_counts()) == 1:  
        return D.iloc[0, -1]
    
    if not attributes:
        return D.iloc[:, -1].mode()[0] 
    
    gains = {attribute: info_gain(D, attribute) for attribute in attributes}
    best_attribute = max(gains, key=gains.get)
    
    tree = {best_attribute: {}}
    
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]
    
    for value in D[best_attribute].unique():
        subset = D[D[best_attribute] == value]
        tree[best_attribute][value] = id3(subset, remaining_attributes)
    
    return tree

C4.5

In [7]:
# C4.5 - Gain Ratio
def split_info(D, attribute):
    values = D[attribute].unique()
    total_size = len(D)
    split_info_value = 0
    for value in values:
        subset_size = len(D[D[attribute] == value])
        proportion = subset_size / total_size
        if proportion > 0:
            split_info_value -= proportion * np.log2(proportion)
    return split_info_value

def gain_ratio(D, attribute):
    gain = info_gain(D, attribute)
    split_info_value = split_info(D, attribute)
    if split_info_value == 0:
        return 0
    return gain / split_info_value

In [8]:
def c45(D, attributes):
    if len(D.iloc[:, -1].value_counts()) == 1:
        return D.iloc[0, -1]
    
    if not attributes:
        return D.iloc[:, -1].mode()[0]
    
    # Find the best attribute based on gain ratio
    gain_ratios = {attribute: gain_ratio(D, attribute) for attribute in attributes}
    best_attribute = max(gain_ratios, key=gain_ratios.get)
    
    tree = {best_attribute: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]
    
    for value in D[best_attribute].unique():
        subset = D[D[best_attribute] == value]
        tree[best_attribute][value] = c45(subset, remaining_attributes)
    
    return tree

CART

In [9]:
# CART - Gini Index
def gini(D):
    class_counts = D.iloc[:, -1].value_counts()
    probs = class_counts / len(D)
    return 1 - np.sum(probs ** 2)

def gini_for_attribute(D, attribute):
    values = D[attribute].unique()
    weighted_gini_sum = 0
    for value in values:
        subset = D[D[attribute] == value]
        weighted_gini_sum += (len(subset) / len(D)) * gini(subset)
    return weighted_gini_sum

In [10]:
def cart(D, attributes):
    if len(D.iloc[:, -1].value_counts()) == 1:
        return D.iloc[0, -1]
    
    if not attributes:
        return D.iloc[:, -1].mode()[0]
    
    # Find the best attribute based on Gini index
    gini_values = {attribute: gini_for_attribute(D, attribute) for attribute in attributes}
    best_attribute = min(gini_values, key=gini_values.get)
    
    tree = {best_attribute: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]
    
    for value in D[best_attribute].unique():
        subset = D[D[best_attribute] == value]
        tree[best_attribute][value] = cart(subset, remaining_attributes)
    
    return tree

In [11]:
attributes = list(df.columns[:-1])  # All columns except the last one (the target column)

id3_tree = id3(df, attributes.copy())
print("ID3 Tree:")
print(id3_tree)

c45_tree = c45(df, attributes.copy())
print("\nC4.5 Tree:")
print(c45_tree)

cart_tree = cart(df, attributes.copy())
print("\nCART Tree:")
print(cart_tree)

ID3 Tree:
{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

C4.5 Tree:
{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

CART Tree:
{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}


If the Outlook is Sunny:

If Humidity is High, the decision is No (do not play tennis).

If Humidity is Normal, the decision is Yes (play tennis).

If the Outlook is Overcast, the decision is always Yes (play tennis).

If the Outlook is Rain:

If Wind is Weak, the decision is Yes (play tennis).

If Wind is Strong, the decision is No (do not play tennis).